# Convolutional Sentiment Analysis

- Dataset: Moview review (rt-polarity.neg, rt-polarity.pos)
    - Source: https://github.com/yoonkim/CNN_sentence
    
- Code reference code
    - https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

- Using kaggle kernel
- Pytorch

**> Reference paper**
- Convolutional Neural Networks for Sentence Classification (EMNLP 2014)
- https://arxiv.org/abs/1408.5882

## import modules

In [1]:
import numpy as np
from collections import defaultdict
import sys, re
import pandas as pd
import os
import pickle
import codecs
import random
import json
import torch
# from torchtext import data
from torchtext.data import TabularDataset, Field, LabelField, BucketIterator
# from torchtext import datasets

from tqdm import tqdm_notebook

## preprocess and load data

In [2]:
## preprocess from original source code: https://github.com/yoonkim/CNN_sentence

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [3]:
def build_data(data_dir = '/kaggle/input/polaritydata/', cv=10):
  
    revs = []
    pos = codecs.open(os.path.join(data_dir, "rt-polarity.pos"), "r", encoding='utf-8', errors='ignore').read()
    neg = codecs.open(os.path.join(data_dir, "rt-polarity.neg"), "r", encoding='utf-8', errors='ignore').read()
    pos_list = [clean_str(sent) for sent in pos.split('\n')[:-1]]
    neg_list = [clean_str(sent) for sent in neg.split('\n')[:-1]]
    print('pos len', len(pos_list))
    print('neg len', len(neg_list))
    print('total len',len(pos_list) + len(neg_list) )
  
    for sent in pos_list:
        datum = {'label': 1,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                 }
        revs.append(datum)

    for sent in neg_list:
        datum = {'label': 0,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                }
        revs.append(datum)

    word_to_idx = {'@pad': 0}

    for sent in pos_list + neg_list:
        for word  in sent.split():
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)



    print(f'num of total data: {len(revs)}')
    print(f'num of vocab: {len(word_to_idx)}')


    df = pd.DataFrame(revs) 
    df.to_csv('./polarity_df.csv', index = False, encoding = 'utf-8')
    

    print('save the data')
    
    return revs, word_to_idx

In [4]:
revs, word_to_idx = build_data()

pos len 5331
neg len 5331
total len 10662
num of total data: 10662
num of vocab: 18778
save the data


In [5]:
# same seed with paper

SEED = 3435

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
TEXT = Field(sequential = True, # text: sequential data
             tokenize = str.split, 
             batch_first = True, 
             fix_length = 56, # padding size: max length of data text
             lower = True)
LABEL = LabelField(sequential = False,
                   dtype = torch.float)

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
dataset = TabularDataset(path='./polarity_df.csv', skip_header = True,
    format='csv', fields=[('label', LABEL), ('text', TEXT)]
                        )
# train, test split
train_data, test_data = dataset.split(random_state = random.seed(SEED),split_ratio=0.9)
print('train_data', len(train_data))
print('test_data', len(test_data))

/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


train_data 9596
test_data 1066


In [8]:
# show a example
print(vars(train_data[0]))

{'label': '1', 'text': ['writer', 'director', 'burger', 'imaginatively', 'fans', 'the', 'embers', 'of', 'a', 'dormant', 'national', 'grief', 'and', 'curiosity', 'that', 'has', 'calcified', 'into', 'chronic', 'cynicism', 'and', 'fear']}


## loading pretrained word2vec embedding

In [9]:
from gensim.models import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('../input/word2vec/GoogleNews-vectors-negative300.bin', binary = True)

In [10]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)
print(len(TEXT.vocab))
print(LABEL.vocab.stoi)

17879
defaultdict(None, {'0': 0, '1': 1})


In [11]:
# for pretrained embedding vectors

EMBEDDING_DIM = 300
w2v_vectors = []
for token, idx in tqdm_notebook(TEXT.vocab.stoi.items()):
    # if word in word2vec vocab -> replace with pretrained word2vec
    if token in w2v.wv.vocab.keys():
        w2v_vectors.append(torch.FloatTensor(w2v[token]))
    # oov -> randomly initialized uniform distribution
    else: 
        w2v_vectors.append(torch.distributions.Uniform(-0.25, +0.25).sample((EMBEDDING_DIM,)))
print(len(w2v_vectors))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  



17879


In [13]:
print('unk token')
print(w2v_vectors[0][:10])
print('general words')
print(w2v_vectors[10][:10])

unk token
tensor([ 0.0410,  0.0286, -0.2246,  0.0923,  0.1683, -0.1634,  0.0673,  0.2371,
         0.2392, -0.1895])
general words
tensor([ 0.0845, -0.0004,  0.0532,  0.0996, -0.1484, -0.0625,  0.0564, -0.0649,
         0.0659,  0.0188])


In [14]:
# gpu setting
BATCH_SIZE = 50
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# making iterators
train_iterator,  test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle = True)

cuda:0


/opt/conda/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [15]:
for a , b in train_iterator:
    print('train text')
    print(a)
    print(a.size())
    print()
    print('train label')
    print(b)
    print(b.size())
    break

/opt/conda/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


train text
tensor([[ 621, 1948,    3,  ...,    1,    1,    1],
        [  10,  225,    4,  ...,    1,    1,    1],
        [  22,  463,    2,  ...,    1,    1,    1],
        ...,
        [ 306,   14,   26,  ...,    1,    1,    1],
        [  21, 2075, 3904,  ...,    1,    1,    1],
        [   4, 2628,   17,  ...,    1,    1,    1]], device='cuda:0')
torch.Size([50, 56])

train label
tensor([1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
torch.Size([50])


## model

In [16]:
import torch.nn as nn
import torch.nn.functional as F


## CNN-1D modeling

modeling source code: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb


In [17]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs,
                                             padding=(fs-1))
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
#         print('text', text.shape)
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
#         print('embedded', embedded.shape)
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
#         print('after permute embedded', embedded.shape)
        conved = [F.relu(conv(embedded)) for conv in self.convs]
#         print('conved', len(conved))
#         print('** conved[0]', conved[0].shape)
#         print('** conved[1]', conved[1].shape)
#         print('** conved[2]', conved[2].shape)
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
#         print('pooled', len(pooled))
#         print('** pooled[0]',pooled[0].shape)
#         print('** pooled[1]',pooled[1].shape)
#         print('** pooled[2]',pooled[2].shape)
       
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
#         print('cat', cat.shape)
        #cat = [batch size, n_filters * len(filter_sizes)]
#         print('self.fc(cat)', self.fc(cat).shape)
#         print('***********************************************')
        return self.fc(cat)

In [18]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
print('INPUT_DIM', INPUT_DIM)
print('EMBEDDING_DIM', EMBEDDING_DIM)
print('PAD_IDX', PAD_IDX)
print('UNK_IDX', UNK_IDX)

INPUT_DIM 17879
EMBEDDING_DIM 300
PAD_IDX 1
UNK_IDX 0


In [19]:
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,724,301 trainable parameters


In [21]:
# for check up (before applying word2vec)
test = 'good'
test_indx = TEXT.vocab.stoi[test]
print(f'test string; {test} \nTest string: {test_indx}')

original_vector = w2v[test]
torch_model_vector = model.embedding(torch.tensor([test_indx]))[0]
print(f'before apply\n origianl: {original_vector[:3]} \n torch_vector: {torch_model_vector[:3]}')

test string; good 
Test string: 54
before apply
 origianl: [ 0.04052734  0.0625     -0.01745605] 
 torch_vector: tensor([ 1.0903, -0.5548, -0.9899], grad_fn=<SliceBackward>)


In [22]:
# apply w2v
TEXT.vocab.set_vectors(TEXT.vocab.stoi, w2v_vectors, EMBEDDING_DIM)
pretrained_embeddings = torch.FloatTensor(TEXT.vocab.vectors)
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0410,  0.0286, -0.2246,  ...,  0.2070,  0.0286, -0.0248],
        [-0.2156,  0.1972,  0.1520,  ..., -0.1784, -0.0852, -0.0421],
        [ 0.0801,  0.1050,  0.0498,  ...,  0.0037,  0.0476, -0.0688],
        ...,
        [ 0.1855,  0.1689,  0.0027,  ..., -0.3301,  0.0500, -0.2119],
        [ 0.0632,  0.0088,  0.2260,  ..., -0.1238, -0.1564,  0.2392],
        [ 0.1859,  0.0750,  0.0866,  ...,  0.1684, -0.2417,  0.1577]])

In [23]:
# for check up (after applying word2vec)
torch_model_vector = model.embedding(torch.tensor([test_indx]))[0]
print(f'after apply\n origianl: {original_vector[:3]} \n torch_vector: {torch_model_vector[:3]}')

after apply
 origianl: [ 0.04052734  0.0625     -0.01745605] 
 torch_vector: tensor([ 0.0405,  0.0625, -0.0175], grad_fn=<SliceBackward>)


In [24]:
# padding -> zero vectors
model.embedding.weight.data[PAD_IDX]= torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [26]:
import torch.optim as optim

# optimizer = optim.Adadelta(model.parameters(), weight_decay=0.95, rho=0.95,eps=1e-6)
optimizer = optim.Adadelta(model.parameters(), rho=0.95)

# BCEWithLogitsLoss automatically does softmax function
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [27]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [49]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
    #  l2 norm (weight contraints): 3
        for name, param in model.named_parameters():
            if 'fc.weight' in name:
                max_val = 3
                norm = param.norm( dim=0, keepdim=True)
                desired = torch.clamp(norm, 0, max_val)
                scale = desired / (1e-7 + norm)
                param = param * scale
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [50]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [51]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [52]:
N_EPOCHS = 10

# best_test_loss = float('inf')
best_test_acc = -float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        torch.save(model.state_dict(), 'tut4-model.pt')

    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Test. Loss: {test_loss:.3f} |  Val. Acc: {test_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.559 | Train Acc: 70.48%
	 Test. Loss: 0.460 |  Val. Acc: 79.33%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.417 | Train Acc: 81.14%
	 Test. Loss: 0.431 |  Val. Acc: 79.86%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.330 | Train Acc: 85.80%
	 Test. Loss: 0.452 |  Val. Acc: 79.25%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.250 | Train Acc: 89.74%
	 Test. Loss: 0.444 |  Val. Acc: 80.25%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.180 | Train Acc: 93.05%
	 Test. Loss: 0.492 |  Val. Acc: 79.67%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.114 | Train Acc: 96.17%
	 Test. Loss: 0.538 |  Val. Acc: 80.43%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.075 | Train Acc: 97.65%
	 Test. Loss: 0.589 |  Val. Acc: 79.50%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.046 | Train Acc: 98.69%
	 Test. Loss: 0.655 |  Val. Acc: 80.50%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.026 | Train Acc: 99.49%
	 Test. Loss: 0.728 |  Val. Acc: 80.15%
Epoch: 10 | Epoch T

In [53]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.679 | Test Acc: 80.11%
